# Import des modules

In [132]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from sklearn.cluster import KMeans
import json

# Lecture des données

In [135]:
db = pd.read_csv("../Ars_data_v_2/output_data.csv", encoding="utf-8", low_memory=False)
db.drop(db.columns[0], axis=1, inplace=True)

# Vue sur les données

In [77]:
db.head(5)

,Nom_com,RFM16,TN16,PM16,NSP19,DP16,PGP19,PGP19_>60,PC16_15_24,PC16_24_64,PPop16_<18,PPop16_>65,PRP16,code_dep,code_commune_INSEE,nom_departement,nom_region,code_postal,code_region
0,L'Abergement-Clémenciat,23440.0,9.1,767.0,0,49.0,0.000000,NaN,0.010430,0.043025,0.252934,0.186441,0.026144,01,01001,Ain,Auvergne-Rhône-Alpes,1400.0,84.0
1,L'Abergement-de-Varey,24214.0,24.7,243.0,0,26.6,0.000000,NaN,0.016461,0.041152,0.255144,0.181070,0.000000,01,01002,Ain,Auvergne-Rhône-Alpes,1640.0,84.0
2,Ambérieu-en-Bugey,19450.0,15.6,14081.0,1,575.0,0.000284,0.190476,0.018323,0.076699,0.239259,0.164903,0.205544,01,01004,Ain,Auvergne-Rhône-Alpes,1500.0,84.0
3,Ambérieux-en-Dombes,23280.0,8.4,1671.0,0,104.1,0.000598,0.500000,0.014961,0.040694,0.241173,0.153800,0.048438,01,01005,Ain,Auvergne-Rhône-Alpes,1330.0,84.0
4,Ambléon,NaN,18.2,110.0,0,18.2,0.000000,NaN,0.027273,0.072727,0.136364,0.236364,0.000000,01,01006,Ain,Auvergne-Rhône-Alpes,1300.0,84.0


In [17]:
db.columns

Index(['Nom_com', 'RFM16', 'TN16', 'PM16', 'NSP19', 'DP16', 'PGP19',
       'PGP19_>60', 'PC16_15_24', 'PC16_24_64', 'PPop16_<18', 'PPop16_>65',
       'PNRP16', 'code_dep', 'code_commune_INSEE', 'nom_departement',
       'nom_region', 'code_postal', 'code_region'],
      dtype='object')

In [18]:
db.describe()

,RFM16,TN16,PM16,NSP19,DP16,PGP19,PGP19_>60,PC16_15_24,PC16_24_64,PPop16_<18,PPop16_>65,PNRP16,code_postal,code_region
count,29394.00000,33158.000000,33164.000000,33178.000000,33163.000000,33158.000000,8110.000000,33158.000000,33158.000000,33158.000000,33158.000000,33078.000000,33176.000000,33176.000000
mean,20802.71273,9.347449,1463.503106,0.163874,153.979314,0.000108,0.327509,0.010492,0.049889,0.216025,0.212755,0.032263,46493.196678,52.260248
std,3620.13035,6.648896,4787.584346,0.839036,715.831508,0.000384,0.355695,0.007858,0.021618,0.050916,0.075225,0.060830,25588.586500,24.272476
min,8218.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1000.000000,1.000000
25%,18504.00000,5.400000,191.000000,0.000000,18.200000,0.000000,0.000000,0.005728,0.036265,0.186632,0.160000,0.000000,25147.500000,28.000000
50%,20236.00000,9.000000,430.000000,0.000000,39.600000,0.000000,0.250000,0.009720,0.047319,0.219992,0.200299,0.000000,48170.000000,44.000000
75%,22418.00000,12.500000,1060.000000,0.000000,92.550000,0.000000,0.500000,0.014184,0.060741,0.249666,0.253968,0.041096,67140.000000,76.000000
max,50092.00000,200.000000,155090.000000,41.000000,26224.000000,0.011494,1.000000,0.250909,0.333333,0.582391,0.857143,0.689355,97680.000000,94.000000


# Features engineering

In [136]:
code_dep = set(db['code_dep'])
for code in code_dep:
    db_temp = db[db['code_dep'] == code]
    sum_pop = db_temp['PM16'].sum()
    db.loc[(db['code_dep'] == code),'PPMsD16']= db['PM16']/sum_pop
db.drop(["PM16"], axis=1, inplace=True)
db=db.where(pd.notnull(db), None)

# Séparation des données

In [129]:
db.columns

Index(['Nom_com', 'RFM16', 'TN16', 'NSP19', 'DP16', 'PGP19', 'PGP19_>60',
       'PC16_15_24', 'PC16_24_64', 'PPop16_<18', 'PPop16_>65', 'PRP16',
       'code_dep', 'code_commune_INSEE', 'nom_departement', 'nom_region',
       'code_postal', 'code_region', 'PPMsD16'],
      dtype='object')

In [109]:
var_desc = db[["Nom_com","code_dep","code_commune_INSEE",
               "nom_departement","nom_region","code_postal","code_region"]] 
var_num = db[["RFM16","TN16","PPMsD16","NSP19","DP16","PGP19","PGP19_>60",
              "PC16_24_64","PC16_15_24","PPop16_<18","PPop16_>65","PRP16"]]

# Calcule des valeurs à afficher

In [25]:
median = var_num.median()
std = var_num.std()
#np.quantile(a, 0.25, axis=1, keepdims=True)

In [26]:
def calcule_indicateur(df):
    i = 0
    colnames = df.columns
    for colname in colnames:
        df[colname] = df[colname].apply(lambda x: 1 if(x>median[i] + std[i]) else 
                                        (-1 if(x<median[i] - std[i]) else 0))
        i+=1
    return df

In [27]:
display_value = calcule_indicateur(var_num)

C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
col_sum = display_value.sum(axis=1)

In [29]:
final = pd.concat([var_desc,display_value],axis=1)
final['Score'] = col_sum/len(var_num.columns)

Pour Théo (Auto Complétion)

In [12]:
final['Nom_com'].to_csv("liste_des_communes", sep=",")

# Export en json

In [130]:
d = db.set_index("Nom_com").T.to_dict()

C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [137]:
with open('data.json', 'w', encoding='utf8') as file:
     file.write(json.dumps(d, ensure_ascii=False))

# Classification

In [124]:
X = var_num

In [125]:
X

,RFM16,TN16,PPMsD16,NSP19,DP16,PGP19,PGP19_>60,PC16_24_64,PC16_15_24,PPop16_<18,PPop16_>65,PRP16
0,23440,9.1,0.00148472,0,49,0,None,0.0430248,0.0104302,0.252934,0.186441,0.0261438
1,24214,24.7,0.000470389,0,26.6,0,None,0.0411523,0.0164609,0.255144,0.18107,0
2,19450,15.6,0.0272574,1,575,0.000284071,0.190476,0.0766991,0.0183226,0.239259,0.164903,0.205544
3,23280,8.4,0.00323465,0,104.1,0.000598444,0.5,0.0406942,0.0149611,0.241173,0.1538,0.0484375
4,None,18.2,0.000212933,0,18.2,0,None,0.0727273,0.0272727,0.136364,0.236364,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33173,None,None,None,0,None,None,None,None,None,None,None,None
33174,None,None,None,0,None,None,None,None,None,None,None,None
33175,None,None,None,0,None,None,None,None,None,None,None,None
33176,None,None,None,0,None,None,None,None,None,None,None,None


In [118]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').